In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from sklearn import manifold

In [3]:
import torch
import sys
import os
import importlib
import torch
import numpy
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict
import pickle
import random
# from asnorm import *

In [4]:
# make cohort set

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [5]:
sys.path.append('/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_dist')

In [7]:
# from DatasetLoader import loadWAV

In [8]:
SpeakerNetModel = importlib.import_module('models.'+'EPACA-TDNN').__getattribute__('MainModel')

In [9]:
# EPACA-TDNN
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

EPACA-TDNN.py, Embedding size is 192, Channels 1024, Spec_aug False.


In [6]:
model_path = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/multi_gpu_epaca_tdnn_soxaug/model/model000000034.model'

In [7]:
train_list = '/workspace/DATASET/server9_ssd/sdsv21/vox2_trainlist.txt'
# train_list = '/workspace/DATASET/server9_ssd/sdsv21/sdsv21_DA_10960.txt'
# train_list_FA = '/workspace/DATASET/server9_ssd/sdsv21/sdsv21_normal_FA1417.txt'
# train_list_EN = '/workspace/DATASET/server9_ssd/sdsv21/sdsv21_normal_EN9543.txt'
train_path = '/workspace/DATASET/server9_ssd/sdsv21'
test_list = '/workspace/DATASET/server9_ssd/sdsv21/vox_h_triallist.txt'
enroll_list = ''
test_path = '/workspace/DATASET/server9_ssd/sdsv21'
score_file = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/041901/result/eval_scores.txt'
result_path = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/041901/result'
out_path_1 = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/041901/result/eval_scores_as1.txt'

In [ ]:
S.cuda()

In [ ]:
loaded_state = torch.load(model_path, map_location="cuda:0")

In [ ]:
# self_state = self.__model__.module.state_dict()
# loaded_state = torch.load(path, map_location="cuda:%d"%self.gpu)
# loaded_state = torch.load(path, map_location="cpu")
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name

    ## pass spk clf weight
    if '__L__' in name:
        print('pass __L__ classerfier W')
        continue

    ## pass DA weight
    if 'DA_module' in name:
        print('pass DA_module params:'+name)
        continue

    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
        continue

    self_state[name].copy_(param)

In [ ]:
S.eval()

In [ ]:
def loadWAV(filename, max_frames):

    # Maximum audio length
    max_audio = max_frames * 160 + 240

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)

    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]

    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [ ]:
loaded_state['model']['__L__.W'].size()

In [ ]:
W_matrix = loaded_state['model']['__L__.W']

In [ ]:
W_matrix = W_matrix.t()

In [ ]:
W_matrix.size()

In [ ]:
with open(os.path.join(train_path, train_list)) as dataset_file:
    lines = dataset_file.readlines()

dictkeys = list(set([x.split()[0] for x in lines]))
dictkeys.sort()
dictkeys = { key : ii for ii, key in enumerate(dictkeys) }
len_dictkeys_ori = len(dictkeys)
if True:
    len_dictkeys = len(dictkeys)
    for ii, key in enumerate(list(dictkeys.keys())):
        dictkeys[key+'_slow'] = ii + len_dictkeys
        dictkeys[key+'_fast'] = ii + len_dictkeys*2

    assert len(dictkeys) == 3*len_dictkeys_ori

In [ ]:
list(dictkeys.keys())[-1]

In [ ]:
list(dictkeys.keys())[5994-1]

In [ ]:
with open(os.path.join(train_path, train_list)) as dataset_file:
    FA_lines = dataset_file.readlines()
FA_dictkeys = list(set([x.split()[0] for x in FA_lines]))

In [ ]:
len(FA_dictkeys)

In [ ]:
FA_list = []
for i in FA_dictkeys:
    print(dictkeys[i])
    FA_list.append(dictkeys[i])

In [ ]:
FA_list.sort()

In [ ]:
FA_list

In [ ]:
FA_list[-1]

In [ ]:
cohort_FA_spk_dict_mean = {}

In [ ]:
for count, label_num in enumerate(FA_list):
    cohort_FA_spk_dict_mean[label_num] = F.normalize(W_matrix[label_num], p=2, dim=0).detach().cpu()
print(count)

In [ ]:
cohort_FA_spk_dict_mean_nm = cohort_FA_spk_dict_mean

In [ ]:
# test sdsv21

In [ ]:
listfilename = test_list
files = []
with open(listfilename) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split()

        if len(data) == 2: data = [random.randint(0,1)] + data
        
        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [ ]:
len(setfiles)

In [ ]:
test_dict = {}

In [ ]:
for count, line in enumerate(setfiles):
    wavline = line
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline, max_frames=0)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu()

    test_dict[line] = ref_feat
    
    if ((count+1) % 1000) == 0:
        print((count+1)//1000, end='\r')

In [ ]:
len(test_dict)

In [ ]:
test_dict_nm = {}
for i in test_dict:
    test_dict_nm[i] = F.normalize(test_dict[i].squeeze(0), p=2, dim=0)

In [ ]:
### optional
# import pickle
# with open('tmp_chort_dict', 'wb') as f:
#     pickle.dump(cohort_spk_dict, f)
# import pickle
# with open('tmp_test_dict', 'wb') as f:
#     pickle.dump(test_dict, f)

In [ ]:
4708 * 5994 // 100000

In [ ]:
count = 0
cohort2test = {}
for i in cohort_FA_spk_dict_mean_nm:
    cohort_emb = cohort_FA_spk_dict_mean_nm[i]
    for j in test_dict_nm:
        test_emb = test_dict_nm[j]
        score = F.cosine_similarity(cohort_emb, test_emb, dim=0).numpy().astype(numpy.float16)
        cohort2test[str(i)+" "+j] = score

        count += 1
        if (count % 100000) == 0:
            print(count // 100000, end='\r')
                
print(count)

In [ ]:
len(cohort2test)

In [ ]:
cohort2test_save_path = os.path.join(result_path, 'tmp_cohort2test.txt')

In [8]:
# use voxe cohort
result_path = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/041900/result'
cohort2test_save_path = os.path.join(result_path, 'tmp_cohort2test.txt')

In [ ]:
with open(cohort2test_save_path, 'w') as f:
    for count, i in enumerate(cohort2test):
        line = '%s %.4f\n'%(i, cohort2test[i])
        f.write(line)
        if (count % 100000) == 0:
            print(count // 100000, end='\r')

In [ ]:
cohort2test = None

In [ ]:
## optional
# import pickle
# with open('tmp_chort_dict', 'wb') as f:
#     pickle.dump(cohort2test, f)
# import pickle
# with open('tmp_test_dict', 'wb') as f:
#     pickle.dump(test_dict, f)

In [11]:
import torch
import sys
import os
import torch
import numpy
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict

def as_norm_1(file_score_path, out_path, cohort2enroll_path, cohort2test_path, top_num=1000, hold_name=True, dict_sep=' '):
    # operation: norm_score = 0.5* ((score-mean_e_e)/str_e_e+(score-mean_t_t)/str_t_t)
    print('Score norm operation start, method: Adaptive_score_norm_type_1')
    print('')
    
    file_scores_as_norm_path = out_path
    enroll_dict = defaultdict(list)
    test_dict = defaultdict(list)

    with open(cohort2enroll_path, 'r') as f:
        for count, i in enumerate(f):
            cohort_utt, enroll_utt, score = i.strip().split(dict_sep)
            score = float(score)
            enroll_dict[enroll_utt].append([cohort_utt, score])
            if ((count+1) % 100000) == 0:
                print('read c2e:', (count+1)//100000, end='\r')
    
#     with open(cohort2test_path, 'r') as f:
#         for count, i in enumerate(f):
#             cohort_utt, test_utt, score = i.strip().split(dict_sep)
#             score = float(score)
#             test_dict[test_utt].append([cohort_utt, score])
#             if ((count+1) % 100000) == 0:
#                 print('read c2t:', (count+1)//100000, end='\r')
                
        
    mean_e_e = {}
    str_e_e = {}
    mean_t_t = {}
    str_t_t = {}
    
    print('')
    print('calculate enroll statistics')
    print('')
    
    for count, key in enumerate(enroll_dict):
        enroll_dict[key] = sorted(enroll_dict[key],key = lambda x:x[1], reverse = True)
        tmp_score_list = []
        for i in range(top_num):
            tmp_score_list.append(enroll_dict[key][i][1])
        mean_e_e[key] = numpy.mean(tmp_score_list)
        str_e_e[key] = numpy.std(tmp_score_list, ddof=1)
        if ((count+1) % 1000) == 0:
            print('cal e:', (count+1)//1000, end='\r')
            
    mean_t_t = mean_e_e
    str_t_t = str_e_e

#     for count, key in enumerate(test_dict):
#         test_dict[key] = sorted(test_dict[key],key = lambda x:x[1], reverse = True)
#         tmp_score_list = []
#         for i in range(top_num):
#             tmp_score_list.append(test_dict[key][i][1])
#         mean_t_t[key] = numpy.mean(tmp_score_list)
#         str_t_t[key] = numpy.std(tmp_score_list, ddof=1)
#         if ((count+1) % 1000) == 0:
#             print('cal t:', (count+1)//1000, end='\r')

    print('')
    print('Scoring...')
    print('')
    
    file_scores = open(file_score_path)
    with open(file_scores_as_norm_path,'w') as f:
        for count, line in enumerate(file_scores):
            enroll_utt = line.split(' ')[1].strip()
            test_utt = line.split(' ')[2].strip()
            score = float(line.split(' ')[0].strip())
            norm_score = 0.5 * ((score-mean_e_e[enroll_utt])/str_e_e[enroll_utt]+(score-mean_t_t[test_utt])/str_t_t[test_utt])
            if hold_name:
                f.write('%.4f %s %s\n'%(norm_score, enroll_utt, test_utt))
            else:
                f.write('%.4f\n'%(norm_score))
            
            if ((count+1) % 10000) == 0:
                print((count+1)//10000, end='\r')
                
    file_scores.close()
    print('')
    print('Adaptive_score_norm_type_1 is finished')

In [12]:
as_norm_1(score_file, out_path_1, cohort2test_save_path, cohort2test_save_path, top_num=1500, hold_name=True)

Score norm operation start, method: Adaptive_score_norm_type_1

read c2e: 8700
calculate enroll statistics

cal e: 145
Scoring...

55
Adaptive_score_norm_type_1 is finished


In [ ]:
# as_norm_2(score_file, out_path_2, cohort2test, cohort2test, top_num=1000, hold_name=True)

In [ ]:
# get asnormed score_file & scoring